In [5]:
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------

# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np

from time import sleep

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options


In [6]:
df_municipios = pd.read_csv('../apis/datos/sugerencias_lugares_2024.csv')

In [11]:
municipios = df_municipios.sample(5)

In [12]:
municipios.drop(columns=['Unnamed: 0'], inplace=True)

In [13]:
municipios = municipios[['nombre_Municipio']]

In [14]:
lista_municipios = [m[0] for m in municipios.values]

lista_municipios

['Campo Real',
 'Fuente el Saz de Jarama',
 'Madrid',
 'Torrelaguna',
 'Las Rozas de Madrid']

In [16]:
lista_municipios_unidos = ["".join(m[0].split()).lower() for m in municipios.values]

lista_municipios_unidos

['camporeal',
 'fuenteelsazdejarama',
 'madrid',
 'torrelaguna',
 'lasrozasdemadrid']

## Municipio de ejemplo

In [17]:
df_codigos_municipios = pd.DataFrame({'munipio': lista_municipios})

In [18]:
df_codigos_municipios

,munipio
0,Campo Real
1,Fuente el Saz de Jarama
2,Madrid
3,Torrelaguna
4,Las Rozas de Madrid


In [20]:
lista_codigos = []
for muni in lista_municipios_unidos:
    chrome_options = Options()
    chrome_options.add_argument("--incognito")

    driver = webdriver.Chrome(options=chrome_options)
    url_wunder = f"https://www.wunderground.com/weather/es/{muni}"
    driver.get(url_wunder)


    driver.maximize_window()


    iframe_cookies = WebDriverWait(driver, 10).until(EC.presence_of_element_located(('xpath', '//*[@id="sp_message_iframe_1165301"]')))
    driver.switch_to.frame(iframe_cookies)
    try:
        sleep(1)
        print("intento aceptar las cookies")
        driver.find_element("css selector", "#notice > div.message-component.message-row.cta-buttons-container > div.message-component.message-column.cta-button-column.reject-column > button").click()
    except:
        print("Element not found.")


    driver.switch_to.default_content()
    sleep(3)

    driver.find_element('css selector', '#inner-content > div.region-content-top > lib-city-header > div:nth-child(1) > div > div > a.station-name').click()

    sleep(2)
    texto = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located(('xpath', '//*[@id="inner-content"]/div[1]/app-dashboard-header/div[2]/div/div[2]/h1')))

    codigo = texto[0].get_attribute('innerHTML').split(' - ')[1]

    lista_codigos.append(codigo)

    driver.quit()

intento aceptar las cookies
intento aceptar las cookies
intento aceptar las cookies
intento aceptar las cookies
intento aceptar las cookies


In [21]:
lista_codigos

['ISEVIL93', 'ISEVIL93', 'IMCHUECA17', 'ITORRE406', 'ISESANLA2']

In [22]:
df_codigos_municipios['codigos'] = pd.Series(lista_codigos)

In [23]:
df_codigos_municipios

,munipio,codigos
0,Campo Real,ISEVIL93
1,Fuente el Saz de Jarama,ISEVIL93
2,Madrid,IMCHUECA17
3,Torrelaguna,ITORRE406
4,Las Rozas de Madrid,ISESANLA2


In [24]:
df_codigos_municipios.to_csv('datos/codigos_municipios.csv')

In [25]:
df_codigos_municipios = pd.read_csv('datos/codigos_municipios.csv')